In [1]:
# Attention is All You Need paper: https://arxiv.org/abs/1706.03762
# OpenAI GPT-3 paper: https://arxiv.org/abs/2005.14165 
# OpenAI ChatGPT blog post: https://openai.com/blog/chatgpt/

In [2]:
# open the file data/input.txt
with open('data/input.txt', 'r') as file:
    text = file.read()

In [3]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [4]:
# all the unique characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('all the unique characters that appear in the text:', chars)
print('vocab_size:', vocab_size)

all the unique characters that appear in the text: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
vocab_size: 65


In [5]:
# character level tokenization
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
# encode and decode
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# encode the text
test_string = 'hello world'
encoded = encode(test_string)
print(encoded)
# decode the text
decoded = decode(encoded)
print(decoded)

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
# split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest will be validation
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

## x = torch.stack([data[i:i+block_size] for i in ix]) 
## above is equivalent to the following:
## result = []
## for i in ix:
##   result.append(data[i:i+block_size])

# radint is generateing random integers between 0 and len(data) - block_size (how many integers to generate)
def get_batch(split): 
    # generate a small batch of data of inputs x and targets y.
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    # y also need [batch_size] chars since its predicting the next character in each sequence
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-'*100)

for b in range(batch_size):
   for t in range(block_size):
      context = xb[b, :t+1]
      target = yb[b, t]
      print(f"when input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----------------------------------------------------------------------------------------------------
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is

In [20]:
# TODO: get to know more about Byram language model.
import torch 
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx: (B, T)
        # logits: (B, T, C)
        logits = self.token_embedding_table(idx)
        
        if targets is not None:
            # 训练模式：需要计算 loss
            B, T, C = logits.shape
            logits_reshaped = logits.view(B*T, C)
            targets_reshaped = targets.view(B*T)
            loss = F.cross_entropy(logits_reshaped, targets_reshaped)
        else:
            # 生成模式：不需要 loss
            loss = None
        
        # 返回的 logits 保持 3维 (B, T, C)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx: (B, T)
        for _ in range(max_new_tokens):
            # 前向传播，不传 targets，所以不计算 loss
            logits, loss = self(idx)
            # logits 现在是 (B, T, C)
            # 只取最后一个时间步的预测
            logits = logits[:, -1, :]  # (B, C)
            # 转成概率
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # 采样下一个 token
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # 拼接到序列后面
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx
    

model = BigramLanguageModel(vocab_size)
out = model(xb)
print(out)

(tensor([[[ 1.7207, -0.8377,  1.2084,  ..., -0.3139, -0.6936,  1.8132],
         [-1.2295,  0.9852,  0.2771,  ..., -0.3737,  1.7950, -0.1492],
         [-1.3926, -0.7724, -0.0716,  ..., -0.4200,  0.8161,  0.6945],
         ...,
         [ 0.6937,  0.5076,  0.0992,  ..., -1.6324, -1.1533,  0.2920],
         [-0.5025,  1.0331,  0.7041,  ...,  0.8140, -0.4832, -0.9733],
         [-1.2295,  0.9852,  0.2771,  ..., -0.3737,  1.7950, -0.1492]],

        [[-1.3243, -0.7549, -1.1034,  ...,  0.7070,  0.0033, -0.4456],
         [-0.5436, -0.7363,  0.1037,  ..., -0.7664,  0.8904,  1.8006],
         [-0.6434,  0.5420, -1.2764,  ..., -0.6016,  0.0989,  0.0179],
         ...,
         [-0.5025,  1.0331,  0.7041,  ...,  0.8140, -0.4832, -0.9733],
         [ 1.3988,  0.0301, -1.4062,  ...,  1.9395, -2.0132,  0.7392],
         [-1.3926, -0.7724, -0.0716,  ..., -0.4200,  0.8161,  0.6945]],

        [[-0.2419,  1.3309,  1.1467,  ..., -1.5730, -0.8000,  0.8150],
         [-1.3926, -0.7724, -0.0716,  ..., -

In [21]:
#
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


!,kMmQEFpWzTiiGeybNr:tCVHgBUYaiCgx!-fdgcEYn?-axKuVNWa.Hwz3!plI 3,p qe!a
eeOkvqHg:!yLr&k:qsvJkMaV$vCP


In [22]:
# create  a simple pytorch optimizer 
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [24]:
batch_size = 32
for steps in range(10000): 
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 100 == 0:
        print(f"step {steps}: loss {loss.item()}")

print(loss.item())

step 0: loss 4.680703163146973
step 100: loss 4.516878604888916
step 200: loss 4.449557781219482
step 300: loss 4.208817958831787
step 400: loss 4.228031158447266
step 500: loss 4.170998573303223
step 600: loss 3.949309825897217
step 700: loss 3.8455967903137207
step 800: loss 3.7703144550323486
step 900: loss 3.798882246017456
step 1000: loss 3.6300606727600098
step 1100: loss 3.518342971801758
step 1200: loss 3.446146249771118
step 1300: loss 3.3913745880126953
step 1400: loss 3.359435796737671
step 1500: loss 3.305678367614746
step 1600: loss 3.3360655307769775
step 1700: loss 3.2590994834899902
step 1800: loss 3.1570487022399902
step 1900: loss 3.1991255283355713
step 2000: loss 3.010910749435425
step 2100: loss 3.13883113861084
step 2200: loss 2.9641757011413574
step 2300: loss 2.9260940551757812
step 2400: loss 2.7834079265594482
step 2500: loss 2.845998525619507
step 2600: loss 2.842567205429077
step 2700: loss 2.8240392208099365
step 2800: loss 2.8292832374572754
step 2900: los

In [26]:
#
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=600)[0].tolist()))


'saithe mas?
ARe oce hof mod arede.

O mam to
T:
WAno hund w FLedst anggt, te ht, bl:
TIUFListoreg RUEOMPo t.
OLar. hin, d h thuch nomfilinor,


Thee t we t'shmshr t s berir n-crpt g s domidiflat ghe pendasthe, otere; war.
I yo bed, lobtom; tndor f c
ICKI yofo, bPUDUCAl' O:
Whened d brcrashe sourestt o rdin ent, iththar lind theldwe erereshimungan? aghellie k'eshisundat, s midit soberay thenteviofou whasin, hy, sousha thoullowild, th mepoor; IAju od ud mee t t by, ks ge my bl YO, s t mp thy,
Whe; pshiede CHa's
Maker'
fe menght che ie ay seshe Ind, pef h ndeathedLIfar

IAnerror irinole; V: me r
